In [ ]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [ ]:
import geemap
import ee
Map=geemap.Map()
Map

In [ ]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
# l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate('2020-08-01','2020-10-31').filter(ee.Filter.lt('CLOUD_COVER',20))
# visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}
# # Map.addLayer(l8, visParams,'image mosaic')
# print(l8.aggregate_array('system:index').getInfo())

# landsat5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa'])
# l5 = landsat5.filterBounds(roi).filterDate('2000-08-01','2000-10-31').filter(ee.Filter.lt('CLOUD_COVER',10))
# print(l5.aggregate_array('system:index').getInfo())

# a = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_122033_20200828')
# visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}
# Map.addLayer(a, visParams,'a')

## 循环，生成总的样本集

In [ ]:
## 异常值去除
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
#### cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
#### k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

In [ ]:
# region = 'tibet'
# sample_region = 'TIBET'
# for year in range(2000,2021,1):
#     print(year)
#     sw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_season' + str(year)+'_v2').randomColumn('random',1,'uniform').sort('random').limit(500)
#     print('抽样的季节性水体： ',sw.size().getInfo())

In [ ]:
# region = 'daqing'
# sample_region = 'DAQING'
# region = 'tianjin'
# sample_region = 'TIANJIN'
# region = 'wuhan'
# sample_region = 'WUHAN'

In [ ]:
totalSamples = ee.FeatureCollection([])
for year in range(1999,2021,5):
    print(year)
    ## 永久性水体样本和非水体样本，抽样
    pw_nw_n = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_noseason' + str(year))
    pw_nw = pw_nw_n.map(lambda i : i.set({'AWEI':ee.Number(i.get('AWEI')).divide(10000)}))
    permanent_water = pw_nw.filter(ee.Filter.eq('waterclass',1)).randomColumn('random',1,'uniform').sort('random').limit(500)
    no_water = pw_nw.filter(ee.Filter.eq('waterclass',0)).randomColumn('random',1,'uniform').sort('random').limit(3000)
    print('抽样的永久性水体： ',permanent_water.size().getInfo())
    print('抽样的非水体： ',no_water.size().getInfo())
    ## 季节性样本，抽样
    # sww = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_season' + str(year)+'_v2')
    sww = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_season' + str(year))
    sw = sww.map(lambda i : i.set({'AWEI':ee.Number(i.get('AWEI')).divide(10000)})).randomColumn('random',1,'uniform').sort('random').limit(500)
    print('抽样的季节性水体： ',sw.size().getInfo())
    
    ## 季节性样本，分类
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    cluster_samples = ee.FeatureCollection([permanent_water,no_water]).flatten()
    clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
    q = ee.FeatureCollection(sw.cluster(clusterer))
    q1 = q.filter(ee.Filter.eq('cluster',1))
    q0 = q.filter(ee.Filter.eq('cluster',0))
    b1 = ee.FeatureCollection(q1).filter(ee.Filter.lt('AWEI',0)).size()
    b2 = ee.FeatureCollection(q1).filter(ee.Filter.gt('AWEI',0)).size()
    c1 = ee.FeatureCollection(q0).filter(ee.Filter.lt('AWEI',0)).size()
    c2 = ee.FeatureCollection(q0).filter(ee.Filter.gt('AWEI',0)).size()
    qq1 = ee.Algorithms.If(ee.Number(b1).lt(ee.Number(b2)),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
    qq0 = ee.Algorithms.If(ee.Number(c1).lt(ee.Number(c2)),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
    seasonWater = ee.FeatureCollection([qq1,qq0]).flatten()
    ## 合并
    total_Samples = ee.FeatureCollection([permanent_water,no_water,seasonWater]).flatten()
    print("筛选前，剩余的样本数： ",total_Samples.size().getInfo())
    ## 异常值去除
    filtered_samples = k_mean(total_Samples)
    print("筛选后，剩余的样本数： ",filtered_samples.size().getInfo())
    print('no-water')
    print(filtered_samples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.lt('AWEI',0)).size().getInfo())
    print(filtered_samples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.gt('AWEI',0)).size().getInfo())
    print('water')
    print(filtered_samples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.lt('AWEI',0)).size().getInfo())
    print(filtered_samples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.gt('AWEI',0)).size().getInfo())
    totalSamples = totalSamples.merge(filtered_samples)
    print("总样本： ",totalSamples.size().getInfo())

In [ ]:
dataset_id = region + '_kmean100_9920_v2'
assetID = 'users/311605001111/SAMPLE_' + sample_region + '/' + dataset_id
task = ee.batch.Export.table.toAsset(**{
    'collection': totalSamples,
    'description': dataset_id,
    'assetId': assetID
})
task.start()

## 分年份生成样本集

In [ ]:
## 异常值去除
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
#### cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
#### k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

In [ ]:
# region = 'daqing'
# sample_region = 'DAQING'
# region = 'tianjin'
# sample_region = 'TIANJIN'
region = 'wuhan'
sample_region = 'WUHAN'

In [ ]:
for year in range(1999,2021,5):
    print(year)
    ## 永久性水体样本和非水体样本，抽样
    pw_nw_n = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_noseason' + str(year))
    pw_nw = pw_nw_n.map(lambda i : i.set({'AWEI':ee.Number(i.get('AWEI')).divide(10000)}))
    permanent_water = pw_nw.filter(ee.Filter.eq('waterclass',1)).randomColumn('random',1,'uniform').sort('random').limit(500)
    no_water = pw_nw.filter(ee.Filter.eq('waterclass',0)).randomColumn('random',1,'uniform').sort('random').limit(3000)
    print('抽样的永久性水体： ',permanent_water.size().getInfo())
    print('抽样的非水体： ',no_water.size().getInfo())
    ## 季节性样本，抽样
    # sww = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_season' + str(year)+'_v2')
    sww = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_season' + str(year))
    sw = sww.map(lambda i : i.set({'AWEI':ee.Number(i.get('AWEI')).divide(10000)})).randomColumn('random',1,'uniform').sort('random').limit(500)
    print('抽样的季节性水体： ',sw.size().getInfo())
    
    ## 季节性样本，分类
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    cluster_samples = ee.FeatureCollection([permanent_water,no_water]).flatten()
    clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
    q = ee.FeatureCollection(sw.cluster(clusterer))
    q1 = q.filter(ee.Filter.eq('cluster',1))
    q0 = q.filter(ee.Filter.eq('cluster',0))
    b1 = ee.FeatureCollection(q1).filter(ee.Filter.lt('AWEI',0)).size()
    b2 = ee.FeatureCollection(q1).filter(ee.Filter.gt('AWEI',0)).size()
    c1 = ee.FeatureCollection(q0).filter(ee.Filter.lt('AWEI',0)).size()
    c2 = ee.FeatureCollection(q0).filter(ee.Filter.gt('AWEI',0)).size()
    qq1 = ee.Algorithms.If(ee.Number(b1).lt(ee.Number(b2)),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
    qq0 = ee.Algorithms.If(ee.Number(c1).lt(ee.Number(c2)),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
    seasonWater = ee.FeatureCollection([qq1,qq0]).flatten()
    ## 合并
    total_Samples = ee.FeatureCollection([permanent_water,no_water,seasonWater]).flatten()
    print("筛选前，剩余的样本数： ",total_Samples.size().getInfo())
    ## 异常值去除
    filtered_samples = k_mean(total_Samples)
    print("筛选后，剩余的样本数： ",filtered_samples.size().getInfo())
    dataset_id = region + str(year)
    assetID = 'users/311605001111/SAMPLE_' + sample_region + '/' + dataset_id
    task = ee.batch.Export.table.toAsset(**{
        'collection': filtered_samples,
        'description': dataset_id,
        'assetId': assetID
    })
    task.start()

### 验证

In [ ]:
region = 'tibet'
sample_region = 'TIBET'
totalSamples = ee.FeatureCollection([])
for year in range(1999,2021,5):
    print(year)
    pw_nw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + str(year))
    print('xxxxxx： ',pw_nw.size().getInfo())
    totalSamples = totalSamples.merge(pw_nw)
    print("总样本： ",totalSamples.size().getInfo())

# 汇总

In [ ]:
# region = 'daqing'
# sample_region = 'DAQING'
# region = 'tianjin'
# sample_region = 'TIANJIN'
# region = 'wuhan'
# sample_region = 'WUHAN'
# region = 'prd'
# sample_region = 'PRD'
region = 'tibet'
sample_region = 'TIBET'

In [ ]:
totalSamples = ee.FeatureCollection([])
for year in range(1999,2021,5):
    print(year)
    ## 永久性水体样本和非水体样本，抽样
    samples = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + str(year))
    totalSamples = totalSamples.merge(samples)
    print("总样本： ",totalSamples.size().getInfo())

In [ ]:
dataset_id = region + '_kmean100_9920_v2'
assetID = 'users/311605001111/SAMPLE_' + sample_region + '/' + dataset_id
task = ee.batch.Export.table.toAsset(**{
    'collection': totalSamples,
    'description': dataset_id,
    'assetId': assetID
})
task.start()